Task04 特征工程

构造监督数据集这里，看文档的描述看的不太懂。我理解是召回和排序两个阶段

1. 基于日志数据，获得用户的最后一次点击操作和用户的历史点击；
2. 基于用户的历史行为制作特征，这个会用到用户的历史点击表，最后的召回列表，文字的信息表和embedding向量
3. 制作标签，形成最后的监督学习数据集

In [1]:
#导包
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import gc, os
import logging
import time
import lightgbm as lgb
from gensim.models import Word2Vec
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 节省内存的一个函数
# 减少内存
def reduce_mem(df):
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'.format(end_mem,
                                                                                                           100*(start_mem-end_mem)/start_mem,
                                                                                                           (time.time()-starttime)/60))
    return df

In [3]:
#定义数据路径
data_path = './data_raw/'
save_path = './tmp_results/'

### 1.数据读取

#### 1.1.训练集和验证集的划分

划分训练集和验证集用于在线下验证模型参数的好坏，

（这里有个疑问，之前的EDA中感觉训练数据和测试数据的分布还是不太一样的，那划分时可不可以将两者合并，再采样出一部分作为线下的验证集）

In [11]:
# all_click_df指的是训练数据
# sample_user_nums 采样作为验证集的数量
def trn_val_split(all_click_df, sample_user_nums):
    all_click = all_click_df
    all_user_ids = all_click.user_id.unique()
    
    # replace = True表示可以重复抽样，反之不可以
    sample_user_ids = np.random.choice(all_user_ids, size = sample_user_nums, replace=False)
    
    click_val = all_click[all_click['user_id'].isin(sample_user_ids)]
    click_trn = all_click[~all_click['user_id'].isin(sample_user_ids)]
    
    #将验证集中的最后一次点击给抽取出来作为答案
    click_val = click_val.sort_values(['user_id', 'click_timestamp'])
    val_ans = click_val.groupby('user_id').tail(1)
    
    click_val = click_val.groupby('user_id').apply(lambda x:x[:-1]).reset_index(drop=True)
    
    #去除val_ans中有些用户只有一次点击的情况，否则这样会在验证模型效果时带来用户冷启动的麻烦
    #保证答案中出现的用户在验证集中出现过
    val_ans = val_ans[val_ans.user_id.isin(click_val.user_id.unique())]
    click_val = click_val[click_val.user_id.isin(val_ans.user_id.unique())]
    
    return click_trn, click_val, val_ans

#### 1.2.获取历史点击和最后一次点击

In [12]:
def get_hist_and_last_click(all_click):
    all_click = all_click.sort_values(by=['user_id','click_timestamp'])
    click_last_df = all_click.groupby('user_id').tail(1)
    
    # 如果用户只有一次点击，那也把这次点击放入历史数据中，默认泄露一下
    def hist_func(user_df):
        if len(user_df)==1:
            return user_df
        else:
            return user_df[:-1]
    
    click_hist_df = all_click.groupby('user_id').apply(hist_func).reset_index(drop=True) 
    return click_hist_df, click_last_df

#### 1.3.读取训练、验证及测试集

`offline=True`为线下训练验证模式，要将训练日志划分为训练集和测试集；`offline=False`为线上提交模式，不需要划分。

In [13]:
def get_trn_val_tst_data(data_path, offline=True):
    if offline:
        click_trn_data = pd.read_csv(data_path+'train_click_log.csv')
        click_trn_data = reduce_mem(click_trn_data)
        click_trn, click_val, val_ans = trn_val_split(all_click_df, sample_user_nums)
    else:
        click_trn = pd.read_csv(data_path+'train_click_log.csv')
        click_trn = reduce_mem(click_trn)
        click_val = None
        val_ans = None
    
    click_tst = pd.read_csv(data_path+'testA_click_log.csv')
    return click_trn, click_val, click_tst, val_ans

#### 1.4.读取召回列表

In [14]:
# 返回多路召回列表或者单路召回列表
def get_recall_list(save_path, single_recall_model=None, multi_recall=False):
    if multi_recall:
        return pickle.load(open(save_path + 'final_recall_items_dict.pkl', 'rb'))
    
    if single_recall_model == 'i2i_itemcf':
        return pickle.load(open(save_path + 'itemcf_recall_dict.pkl', 'rb'))
    elif single_recall_model == 'i2i_emb_itemcf':
        return pickle.load(open(save_path + 'itemcf_emb_dict.pkl', 'rb'))
    elif single_recall_model == 'user_cf':
        return pickle.load(open(save_path + 'youtubednn_usercf_dict.pkl', 'rb'))
    elif single_recall_model == 'youtubednn':
        return pickle.load(open(save_path + 'youtube_u2i_dict.pkl', 'rb'))

#### 1.5.读取各种Embedding

Word2Vec属于word embedding的一种，它有两种经典的算法：
* CBOW（continuous bag of words）：根据中心词周围的上下文单词来预测该词的词向量；
* skip-gram：根据中心词预测周围上下文的词的概率分布

它有两种训练方法：
* negative sampling：通过抽取负样本来定义目标，本质是预测总体类别的一个子集；
* hierarchical softmax：通过霍夫曼树来计算所有词向量的概率来定义目标，本质是把$N$分类问题变成$\log(N)$次二分类。

这里使用`gensim`库来训练word2vec。`gensim`用于从原始的非结构化的文本中，无监督地学习到文本隐层的主题向量表达。它支持包括TF-IDF，LSA，LDA，和word2vec在内的多种主题模型算法，支持流式训练，并提供了诸如相似度计算，信息检索等一些常用任务的API接口。

使用`gensim`训练word2vec的一些重要参数（具体可参考[官方文档](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)）：
* vector_size：Dimensionality of the word vectors.词向量的维度；
* window：Maximum distance between the current and predicted word within a sentence.决定了目标词会与多远距离的上下文产生关系；
* sg：如果是0，则是CBOW模型，如果是1，则是skip-gram模型；
* min_count：Ignores all words with total frequency lower than this.；
* workers：表示训练时使用的线程数量；
* epochs：Number of iterations (epochs) over the corpus.训练时便利整个数据集的次数。

要注意的是：训练时输入的语料库一定要是字符组成的二维码，如：[['北','京','你','好'],['上','海','你','好']]

In [16]:
def train_item_word2vec(click_df, embed_size=16, save_name='item_w2v_emb.pkl', split_cahr=''):
    click_df = click_df.sort_values('click_timestamp')
    #只要转换成字符串才可以进行训练
    click_df['click_article_id'] = click_df['click_article_id'].astype(str)
    
    #转换成句子的形式
    docs = click_df.groupby(['user_id'])['click_article_id'].apply(lambda x: list(x)).reset_index()
    docs = docs['click_article_id'].values.tolist()
    
    #为了方便查看训练的进度，这里设定一个log信息
    logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s', level=logging.INFO)
    
    #这里的参数对训练得到的词向量影响也很大，默认负采样 negative 为5：
    w2v = Word2Vec(docs, size=embed_size, sg=1, 
                   window=5, seed=2020, workers=-1, min_count=1, iter=1)
    
    #保存成字典的形式
    item_w2v_emb_dict = {k:w2v[k] for k in click_df['click_article_id']}
    pickle.dump(item_w2v_emb_dict, open(save_path + 'item_w2v_emb.pkl', 'wb'))
    
    return item_w2v_emb_dict

In [17]:
#可以通过字典查询对于的item的Embedding
def get_embedding(save_path, all_click_df):
    if os.path.exists(save_path + 'item_content_emb.pkl'):
        item_content_emb_dict = pickle.load(open(save_path + 'item_content_emb.pkl', 'rb'))
    else:
        print('item_content_emb.pkl 文件不存在')
    
    if os.path.exists(save_path + 'item_w2v_emb.pkl'):
        item_w2v_emb_dict = pickle.load(open(save_path + 'item_w2v_emb.pkl', 'rb'))
    else:
        item_w2v_emb_dict = train_item_word2vec(all_click_df)
    
    if os.path.exists(save_path + 'item_youtube_emb.pkl'):
        item_youtube_emb_dict = pickle.load(open(save_path + 'item_youtube_emb.pkl', 'rb'))
    else:
        print('item_youtube_emb.pkl 文件不存在')
        
    if os.path.exists(save_path + 'user_youtube_emb.pkl'):
        user_youtube_emb_dict = pickle.load(open(save_path + 'user_youtube_emb.pkl', 'rb'))
    else:
        print('user_youtube_emb.pkl 文件不存在')
    
    return item_content_emb_dict, item_w2v_emb_dict, item_youtube_emb_dict, user_youtube_emb_dict

#### 1.6.读取文章信息，读取数据

In [18]:
def get_article_info_df():
    article_info_df = pd.read_csv(data_path + 'articles.csv')
    article_info_df = reduce_mem(article_info_df)
    
    return article_info_df

In [19]:
#根据需要设定offline的值
click_trn, click_val, click_tst, val_ans = get_trn_val_tst_data(data_path, offline=False)

-- Mem. usage decreased to 23.34 Mb (69.4% reduction),time spend:0.00 min


In [20]:
#获取历史点击和最后一次点击
click_trn_hist, click_trn_last = get_hist_and_last_click(click_trn)

if click_val is not None:
    click_val_hist, click_val_last = click_val, val_ans
else:
    click_val_hist, click_val_last = None, None
    
click_tst_hist = click_tst

### 2.对训练数据做负采样

召回得到的数据是三元组的形式(user, item1, label)的形式，但正负样本比例极度不平衡，负样本数量远大于正样本，毕竟召回那么多个item,用户真正点击的只有1个。因此我们可以先对负样本进行下采样，一方面缓解了正负样本比例的问题，另一方面也减小了我们做排序特征的压力。

做负采样时需注意：
1. 只对负样本进行下采样（也可以考虑对正样本进行扩充）；
2. 负采样之后，保证所有的用户和文章仍然出现在采样之后的数据中；
3. 下采样的比例可以看作是一个超参数，可以根据实际情况认为的控制；
4. 做完负采样之后，更新此刻新的用户召回文字列表，因为后续作特征的时候可用到相对位置的信息。

（其实负采样可以留在后面做完特征再进行，这里由于作排序特征太慢，所以把负采样的环节提到前面。）

简单记录下下面的几个函数的作用和实现方法：
* `recall_dict_2_df(recall_list_dict)`

   比较简单，是将召回列表转换成df的形式，各列分别是`[user_id, sim_item, score]`
* `get_rank_label_df(recall_list_df, label_df, is_test=False)`
  
  `recall_list_df`是上一步得到的召回列表，`label_df`是用户最后一次点击的日志，先将其中的`click_article_id`更命为`sim_item`，以与召回df对应，然后将两个`df`按`label_df[['user_id', 'sim_item', 'click_timestamp']`merge起来，这样如果召回df中命中了用户最后一次点击的物品，那么其时间戳`click_timestamp`就不为空，标签记为1；如果没有命中，则时间戳就为NaN，标签记为0。
  
* `neg_sample_recall_data(recall_items_df, sample_rate=0.001)`

   `recall_items_df`是上一步得到的已经打好标签的数据，先将正、负样本分开来，统计其数量以及正负样本的比例。然后用一个分组采样函数`neg_sample_func(group_df)`，其采样方法是，先根据`group_df`的负样本的个数×采样率，计算一个值，然后规定负采样的数量要大于等于1，小于等于5（认为设定）。
   
   之后分别对用户和物品用这个采样函数采样，每一个用户和每一个物品就是一个`group_df`，取需要个数的负样本，因此这个采样率要根据所选的召回策略及其召回的个数来定，如果我们这里用单路的itemcf召回，它为每个用户召回10个item，采样率取0.01的话每个用户的负样本就只能取1个了，显然是不太合适，所以我觉得若取单路召回，采样率应设大些，取0.5，但这样对基于物品的负采样会有影响吗，比如采样率取0.5时，用户和物品负样本采样合并去重后，数量是正样本数量的8.33倍，或许将用户采样率和物品采样率分开了设置会比较好些？
   
   （这样有的用户就没有正样本了？显然有些用户的召回列表中一个都没命中最后点击的文章）
   
* `get_user_recall_item_label_df(click_trn_hist, click_val_hist, click_tst_hist, click_trn_last, click_val_last, recall_list_df)`

   比较简单，将训练数据的历史点击，最后点击；验证数据的历史点击，最后点击；测试数据输入，返回对应的打好标签，负采样后的数据

#### 2.1.将召回列表转换成df

In [13]:
#将召回列表转换成df的形式
def recall_dict_2_df(recall_list_dict):
    df_row_list = []  #[user, item, socre]
    for user, recall_list in tqdm(recall_list_dict.items()):
        for item, score in recall_list:
            df_row_list.append([user, item, score])
    
    col_names = ['user_id', 'sim_item', 'score']
    recall_list_df = pd.DataFrame(df_row_list, columns=col_names)
    
    return recall_list_df

#### 2.2.给召回数据打标签

In [14]:
def get_rank_label_df(recall_list_df, label_df, is_test=False):
    # 测试集的标签未知， 为了后面的代码统一，这里给一个负数作为测试集的标签
    if is_test:
        recall_list_df['label'] = -1
        return recall_list_df
    
    label_df = label_df.rename(columns={'click_article_id' : 'sim_item'})
    recall_list_df_ = recall_list_df.merge(label_df[['user_id', 'sim_item', 'click_timestamp']], how='left', on=['user_id', 'sim_item'])
    recall_list_df_['label'] = recall_list_df_['click_timestamp'].apply(lambda x : 0.0 if np.isnan(x) else 1.0)
    del recall_list_df_['click_timestamp']
    
    return recall_list_df_

#### 2.3.负采样函数

In [15]:
def neg_sample_recall_data(recall_items_df, sample_rate=0.5):
    pos_data = recall_items_df[recall_items_df['label'] == 1]
    neg_data = recall_items_df[recall_items_df['label'] == 0]
    
    print('pos_data_num:', len(pos_data), 'neg_data_num:', len(neg_data), 'pos/neg:', len(pos_data) / len(neg_data))
    
    #分组采样函数
    def neg_sample_func(group_df):
        neg_num = len(group_df)
        sample_num = max(int(neg_num * sample_rate), 1)   #保证最少有一个
        sample_num = min(sample_num, 5)    #保证最多不超过5个，可根据实际情况进行选择
        return group_df.sample(n=sample_num, replace=True)
    
    #对用户进行负采样，保证所有用户都在采样后的数据中
    neg_data_user_sample = neg_data.groupby('user_id', group_keys=False).apply(neg_sample_func)
    
    #对物品进行负采样，保证所有物品都在采样后的数据中
    neg_data_item_sample = neg_data.groupby('sim_item', group_keys=False).apply(neg_sample_func)
    
    #将上述两种情况下的采样数据合并
    neg_data_new = neg_data_user_sample.append(neg_data_item_sample)
    
    #由于上述两个操作是分开的，可能将两个相同的数据给重复选择了，所以需要对合并后的数据进行去重
    neg_data_new = neg_data_new.sort_values(['user_id', 'score']).drop_duplicates(['user_id', 'sim_item'], keep='last')
    
    #将正负样本数据合并
    data_new = pd.concat([pos_data, neg_data_new], ignore_index=True)
    
    return data_new

#### 2.4.获取负采样之后的可用数据

In [16]:
def get_user_recall_item_label_df(click_trn_hist, click_val_hist, click_tst_hist, click_trn_last, click_val_last, recall_list_df):
    #获取训练数据的召回列表
    trn_user_items_df = recall_list_df[recall_list_df['user_id'].isin(click_trn_hist['user_id'].unique())]
    
    #训练数据打标签
    trn_user_item_label_df = get_rank_label_df(trn_user_items_df, click_trn_last, is_test=False)
    
    #训练数据负采样
    trn_user_item_label_df = neg_sample_recall_data(trn_user_item_label_df)
    
    #验证数据负采样
    if click_val is not None:
        val_user_items_df = recall_list_df[recall_list_df['user_id'].isin(click_val_hist['user_id'].unique())]
        val_user_item_label_df = get_rank_label_df(val_user_items_df, click_val_last, is_test=False)
        val_user_item_label_df = neg_sample_recall_data(val_user_item_label_df)
    else:
        val_user_item_label_df = None
    
    #测试数据不需要进行负采样，直接对所有的召回商品打-1标签
    tst_user_items_df = recall_list_df[recall_list_df['user_id'].isin(click_tst_hist['user_id'].unique())]
    tst_user_item_label_df = get_rank_label_df(tst_user_items_df, None, is_test=True)
    
    return trn_user_item_label_df, val_user_item_label_df, tst_user_item_label_df

In [17]:
#读取召回列表，这里先以单路召回为例,itemcf召回中为每个用户召回10个文章
recall_list_dict = get_recall_list(save_path, single_recall_model='i2i_itemcf')

In [18]:
#将召回数据转换成df
recall_list_df = recall_dict_2_df(recall_list_dict)

100%|██████████| 250000/250000 [00:02<00:00, 102347.36it/s]


In [19]:
recall_list_df[recall_list_df['user_id']==249990]

,user_id,sim_item,score
2499900,249990,162655,2.608960
2499901,249990,156560,1.611808
2499902,249990,300470,1.395827
2499903,249990,158536,1.364642
2499904,249990,160417,1.351503
2499905,249990,16129,1.283932
2499906,249990,158082,1.131684
2499907,249990,158906,1.123988
2499908,249990,160940,0.959119
2499909,249990,156808,0.953004


In [20]:
# 给训练验证数据打标签，并负采样
trn_user_item_label_df, val_user_item_label_df, tst_user_item_label_df = get_user_recall_item_label_df(click_trn_hist, 
                                                                                                       click_val_hist, 
                                                                                                       click_tst_hist, 
                                                                                                       click_trn_last, 
                                                                                                       click_val_last, 
                                                                                                       recall_list_df)

pos_data_num: 95739 neg_data_num: 1904261 pos/neg: 0.05027619638274375


In [21]:
trn_user_item_label_df

,user_id,sim_item,score,label
0,1,63746,0.732900,1.0
1,3,36162,1.986575,1.0
2,5,234481,0.520746,1.0
3,7,211442,1.137394,1.0
4,8,50644,0.965673,1.0
...,...,...,...,...
893367,199998,272883,1.138335,0.0
893368,199999,108855,0.887942,0.0
893369,199999,124228,0.906498,0.0
893370,199999,158794,1.012024,0.0


********************************************
下面再来详细看一下这些函数都生成了什么，可以略过.

In [69]:
trn_user_items_df_1 = recall_list_df[recall_list_df['user_id'].isin(click_trn_hist['user_id'].unique())]

In [70]:
trn_user_items_df_1  #训练日志中共20万个用户，每个用户有10个召回物品

,user_id,sim_item,score
0,0,36162,0.932143
1,0,211442,0.656603
2,0,234481,0.614006
3,0,50644,0.577445
4,0,156279,0.528487
...,...,...,...
1999995,199999,108855,0.887942
1999996,199999,237524,0.870683
1999997,199999,218355,0.824729
1999998,199999,286321,0.801657


In [71]:
click_trn_last

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
1112620,0,157507,1508211702520,4,1,17,1,25,2
1112602,1,63746,1508211346889,4,1,17,1,25,6
1112600,2,168401,1508211468695,4,3,20,1,25,2
1112598,3,36162,1508211389672,4,3,2,1,25,2
1112587,4,39894,1508211655466,4,1,12,1,16,1
...,...,...,...,...,...,...,...,...,...
1069757,199995,277107,1508171185481,4,1,17,1,25,5
729122,199996,277492,1507731677532,4,1,17,1,25,2
8,199997,223931,1507029713061,4,1,17,1,16,1
1037728,199998,236648,1508151216865,4,1,17,1,25,5


In [72]:
trn_user_item_label_df_1 = get_rank_label_df(trn_user_items_df_1, click_trn_last, is_test=False)

In [73]:
trn_user_item_label_df_1

,user_id,sim_item,score,label
0,0,36162,0.932143,0.0
1,0,211442,0.656603,0.0
2,0,234481,0.614006,0.0
3,0,50644,0.577445,0.0
4,0,156279,0.528487,0.0
...,...,...,...,...
1999995,199999,108855,0.887942,0.0
1999996,199999,237524,0.870683,0.0
1999997,199999,218355,0.824729,1.0
1999998,199999,286321,0.801657,0.0


In [77]:
label_df = click_trn_last
recall_list_df = trn_user_items_df_1

label_df = label_df.rename(columns={'click_article_id' : 'sim_item'})
recall_list_df = recall_list_df.merge(label_df[['user_id', 'sim_item', 'click_timestamp']], how='left', on=['user_id', 'sim_item'])

recall_list_df

,user_id,sim_item,score,click_timestamp
0,0,36162,0.932143,NaN
1,0,211442,0.656603,NaN
2,0,234481,0.614006,NaN
3,0,50644,0.577445,NaN
4,0,156279,0.528487,NaN
...,...,...,...,...
1999995,199999,108855,0.887942,NaN
1999996,199999,237524,0.870683,NaN
1999997,199999,218355,0.824729,1.508177e+12
1999998,199999,286321,0.801657,NaN


In [78]:
recall_list_df['label'] = recall_list_df['click_timestamp'].apply(lambda x : 0.0 if np.isnan(x) else 1.0)
del recall_list_df['click_timestamp']

recall_list_df

,user_id,sim_item,score,label
0,0,36162,0.932143,0.0
1,0,211442,0.656603,0.0
2,0,234481,0.614006,0.0
3,0,50644,0.577445,0.0
4,0,156279,0.528487,0.0
...,...,...,...,...
1999995,199999,108855,0.887942,0.0
1999996,199999,237524,0.870683,0.0
1999997,199999,218355,0.824729,1.0
1999998,199999,286321,0.801657,0.0


In [79]:
recall_items_df = recall_list_df

pos_data = recall_items_df[recall_items_df['label'] == 1]
neg_data = recall_items_df[recall_items_df['label'] == 0]
    
print('pos_data_num:', len(pos_data), 'neg_data_num:', len(neg_data), 'pos/neg:', len(pos_data) / len(neg_data))

pos_data_num: 95739 neg_data_num: 1904261 pos/neg: 0.05027619638274375


In [80]:
neg_data

,user_id,sim_item,score,label
0,0,36162,0.932143,0.0
1,0,211442,0.656603,0.0
2,0,234481,0.614006,0.0
3,0,50644,0.577445,0.0
4,0,156279,0.528487,0.0
...,...,...,...,...
1999994,199999,36399,0.905755,0.0
1999995,199999,108855,0.887942,0.0
1999996,199999,237524,0.870683,0.0
1999998,199999,286321,0.801657,0.0


In [83]:
#分组采样函数
sample_rate = 0.5
def neg_sample_func(group_df):
    neg_num = len(group_df)
    sample_num = max(int(neg_num * sample_rate), 1)   #保证最少有一个
    sample_num = min(sample_num, 5)    #保证最多不超过5个，可根据实际情况进行选择
    return group_df.sample(n=sample_num, replace=True)

#对用户进行负采样，保证所有用户都在采样后的数据中
neg_data_user_sample = neg_data.groupby('user_id', group_keys=False).apply(neg_sample_func)

neg_data_user_sample

,user_id,sim_item,score,label
8,0,79851,0.360521,0.0
1,0,211442,0.656603,0.0
1,0,211442,0.656603,0.0
1,0,211442,0.656603,0.0
1,0,211442,0.656603,0.0
...,...,...,...,...
1999982,199998,272883,1.138335,0.0
1999996,199999,237524,0.870683,0.0
1999998,199999,286321,0.801657,0.0
1999998,199999,286321,0.801657,0.0


In [85]:
neg_data_user_sample['user_id'].value_counts()

0         5
14639     5
150062    5
59701     5
152109    5
         ..
51456     4
55554     4
53507     4
61703     4
198303    4
Name: user_id, Length: 200000, dtype: int64

In [91]:
#对物品进行负采样，保证所有物品都在采样后的数据中
sample_rate = 0.4

neg_data_item_sample = neg_data.groupby('sim_item', group_keys=False).apply(neg_sample_func)

neg_data_item_sample 

,user_id,sim_item,score,label
339316,33931,94,0.763304,0.0
329206,32920,94,0.763304,0.0
326888,32688,94,0.686974,0.0
343536,34353,94,0.763304,0.0
1307357,130735,94,0.763304,0.0
...,...,...,...,...
262944,26294,364043,1.789205,0.0
1959226,195922,364043,1.192724,0.0
295717,29571,364043,1.189781,0.0
1631151,163115,364046,3.045757,0.0


In [92]:
neg_data_item_sample['sim_item'].value_counts()

42971     5
166038    5
323721    5
12433     5
118957    5
         ..
297649    1
281257    1
107156    1
217738    1
137219    1
Name: sim_item, Length: 26169, dtype: int64

In [93]:
#将上述两种情况下的采样数据合并
neg_data_new = neg_data_user_sample.append(neg_data_item_sample)

#由于上述两个操作是分开的，可能将两个相同的数据给重复选择了，所以需要对合并后的数据进行去重
neg_data_new = neg_data_new.sort_values(['user_id', 'score']).drop_duplicates(['user_id', 'sim_item'], keep='last')

neg_data_new

,user_id,sim_item,score,label
8,0,79851,0.360521,0.0
7,0,111338,0.375745,0.0
1,0,211442,0.656603,0.0
19,1,106598,0.213678,0.0
17,1,156279,0.229084,0.0
...,...,...,...,...
1999983,199998,86386,1.118216,0.0
1999982,199998,272883,1.138335,0.0
1999998,199999,286321,0.801657,0.0
1999996,199999,237524,0.870683,0.0


In [95]:
#将正负样本数据合并
data_new = pd.concat([pos_data, neg_data_new], ignore_index=True)

data_new

,user_id,sim_item,score,label
0,1,63746,0.732900,1.0
1,3,36162,1.986575,1.0
2,5,234481,0.520746,1.0
3,7,211442,1.137394,1.0
4,8,50644,0.965673,1.0
...,...,...,...,...
889957,199998,86386,1.118216,0.0
889958,199998,272883,1.138335,0.0
889959,199999,286321,0.801657,0.0
889960,199999,237524,0.870683,0.0


分割线内的可略过。
**********************************************************

### 3.做特征

#### 3.1.将召回数据转换成字典

将最终的召回的df数据转换成字典的形式:`{user1: [(sim_item1, score1, label), ....], user2....}`，做排序特征。

In [22]:
def make_tuple_func(group_df):
    row_data=[]
    for name, row_df in group_df.iterrows():
        row_data.append((row_df['sim_item'], row_df['score'], row_df['label']))
        
    return row_data

In [23]:
trn_user_item_label_tuples = trn_user_item_label_df.groupby('user_id').apply(make_tuple_func).reset_index()

In [24]:
trn_user_item_label_tuples_dict = dict(zip(trn_user_item_label_tuples['user_id'], trn_user_item_label_tuples[0]))

In [25]:
trn_user_item_label_tuples

,user_id,0
0,0,"[(324426.0, 0.3309903651165827, 0.0), (79851.0..."
1,1,"[(63746.0, 0.7329003901103845, 1.0), (106598.0..."
2,2,"[(158772.0, 0.5469002679503203, 0.0), (209122...."
3,3,"[(36162.0, 1.9865754276408667, 1.0), (30760.0,..."
4,4,"[(326338.0, 0.4030385725128302, 0.0), (65560.0..."
...,...,...
199995,199995,"[(182513.0, 0.7303524009296932, 0.0), (175040...."
199996,199996,"[(235854.0, 1.3383704374741787, 0.0), (123368...."
199997,199997,"[(223931.0, 0.7649279678821581, 1.0), (342917...."
199998,199998,"[(236648.0, 1.0518565416953483, 1.0), (118316...."


In [28]:
if val_user_item_label_df is not None:
    val_user_item_label_tuples = val_user_item_label_df.groupby('user_id').apply(make_tuple_func).reset_index()
    val_user_item_label_tuples_dict = dict(zip(val_user_item_label_tuples['user_id'], val_user_item_label_tuples[0]))
else:
    val_user_item_label_tuples_dict = None

In [29]:
tst_user_item_label_tuples = tst_user_item_label_df.groupby('user_id').apply(make_tuple_func).reset_index()
tst_user_item_label_tuples_dict = dict(zip(tst_user_item_label_tuples['user_id'], tst_user_item_label_tuples[0]))

#### 3.2.用户历史行为相关特征

对于每个用户召回的每个商品，做特征，方法是：对每个用户，获取最后点击的N个商品的`item_id`，对于该用户的每个召回商品，计算与上面最后N次点击商品的相似度的和（最大，最小，均值），时间差特征，相似性特征，字数差特征，与该用户的相似性特征。

In [51]:
# 下面基于data做历史相关的特征
def create_feature(users_id, recall_list, click_hist_df, articles_info, articles_emb, user_emb=None, N=1):
    '''
    :param articles_emb：文章的embedding向量，这个可以用item_content_emb, item_w2v_emb, item_youtube_emb
    :param user_emb：用户的embedding向量，可以使用之前计算过的user_youtube_emb,但注意这时articles_emb要使用item_youtube_emb，以保证维度一样
    :param N：最近的N次点击，由于TestA日志里很多用户只存在一次历史点击，所以为了不产生空值，N默认取1
    '''
    
    #建立一个二维列表保存结果，后面要转成DataFrame
    all_user_feas = []
    i = 0
    for user_id in tqdm(users_id):
        #该户的最后N次点击
        hist_user_items = click_hist_df[click_hist_df['user_id']==user_id]['click_article_id'][-N:]
        
        #遍历该用户的召回列表
        for rank, (article_id, score, label) in enumerate(recall_list[user_id]):
            #该文章建立时间
            a_create_time = articles_info[articles_info['article_id']==article_id]['created_at_ts'].values[0]
            #该文章的字数
            a_words_count = articles_info[articles_info['article_id']==article_id]['words_count'].values[0]
            
            single_user_fea = [user_id, article_id]
            sim_fea = []
            time_fea = []
            word_fea = []
            
            #计算与最后点击的商品的相似度的和，最大值和最小值，均值
            #遍历用户的最后N次点击文章
            for hist_item in hist_user_items:
                b_create_time = articles_info[articles_info['article_id']==hist_item]['created_at_ts'].values[0]
                b_words_count = articles_info[articles_info['article_id']==hist_item]['words_count'].values[0]
                
                sim_fea.append(np.dot(articles_emb[hist_item], articles_emb[article_id]))
                time_fea.append(abs(a_create_time-b_create_time))
                word_fea.append(abs(a_words_count-b_words_count))
            
            single_user_fea.extend(sim_fea)     #相似性特征
            single_user_fea.extend(time_fea)    #时间差特征
            single_user_fea.extend(word_fea)    #字数差特征
            #这里的最后N次点击只取了一次，那这些统计特征都是一样的吧？
            single_user_fea.extend([max(sim_fea), min(sim_fea), sum(sim_fea), sum(sim_fea)/len(sim_fea)])  #相似性的统计特征
            
            #如果用户emb向量有的话，这里计算该召回文章与用户的相似性特征
            if user_emb:
                single_user_fea.append(np.dot(user_emb[user_id], articles_emb[article_id]))
            
            single_user_fea.extend([score, rank, label])
            all_user_feas.append(single_user_fea)
    
    #定义列名
    id_cols = ['user_id', 'click_article_id']
    sim_cols = ['sim'+str(i) for i in range(N)]
    time_cols = ['time_diff'+str(i) for i in range(N)]
    word_cols = ['word_diff'+str(i) for i in range(N)]
    sat_cols = ['sim_max', 'sim_min', 'sim_sum', 'sim_mean']
    user_item_sim_cols = ['user_item_sim'] if user_emb else []
    user_score_rank_label = ['score', 'rank', 'label']
    cols = id_cols + sim_cols + time_cols + word_cols + sat_cols + user_item_sim_cols + user_score_rank_label
    
    df = pd.DataFrame(all_user_feas, columns=cols)
    
    return df

In [45]:
article_info_df = get_article_info_df()
all_click = click_trn.append(click_tst)
item_content_emb_dict, item_w2v_emb_dict, item_youtube_emb_dict, user_youtube_emb_dict = get_embedding(save_path, all_click)

-- Mem. usage decreased to  5.56 Mb (50.0% reduction),time spend:0.00 min


In [52]:
#获取训练、验证及测试数据中召回列文章相关特征
trn_user_item_feats_df = create_feature(trn_user_item_label_tuples_dict.keys(), trn_user_item_label_tuples_dict, 
                                        click_trn_hist, article_info_df, item_content_emb_dict)

if val_user_item_label_tuples_dict is not None:
    val_user_item_feats_df = create_feature(val_user_item_label_tuples_dict.keys(), val_user_item_label_tuples_dict, 
                                        click_val_hist, article_info_df, item_content_emb_dict)
else:
    val_user_item_feats_df = None

tst_user_item_feats_df = create_feature(tst_user_item_label_tuples_dict.keys(), tst_user_item_label_tuples_dict, 
                                        click_tst_hist, article_info_df, item_content_emb_dict)

100%|██████████| 50000/50000 [20:19<00:00, 41.01it/s]


In [54]:
#保存到本地
trn_user_item_feats_df.to_csv(save_path + 'trn_user_item_feats_df.csv', index=False)

if val_user_item_feats_df is not None:
    val_user_item_feats_df.to_csv(save_path + 'val_user_item_feats_df.csv', index=False)

tst_user_item_feats_df.to_csv(save_path + 'tst_user_item_feats_df.csv', index=False)

### 4.用户和文章特征

#### 4.1.用户相关特征

到这里算是进入真正的特征工程了，既要拼接上已有的特征，也要创造更多的特征。首先来梳理一下已有的特征和可构造特征：

1. 文章自身的特征：文章字数，文章创建时间，文章内容的embedding；
2. 用户点击环境特征，如点击环境，点击设备组，点击操作系统，点击地区，点击城市，点击来源类型等，这些特征在多路回归中并没有使用；
3. 对于用户和商品还可以构造特征：
    * 基于用户的点击文章次数和点击时间（怎么考虑时间？）构造可以表现用户活跃度和特征
    * 基于文章被点击次数和时间构造可以反映文章热度的特征
    * 用户的时间统计特征：根据其点击的历史文章列表的点击时间和文章的创建时间做统计特征，比如求均值，可以来反映用户对于文章时效性的偏好
    * 用户的主题爱好特征：对于用户点击的历史文章主题进行一个统计，然后对于当前文章看看是否属于用户已经点击过的主题
    * 用户的字数爱好特征：对于用户点击的历史文章的字数统计，求一个均值

In [55]:
click_tst.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,249999,160974,1506959142820,4,1,17,1,13,2
1,249999,160417,1506959172820,4,1,17,1,13,2
2,249998,160974,1506959056066,4,1,12,1,13,2
3,249998,202557,1506959086066,4,1,12,1,13,2
4,249997,183665,1506959088613,4,1,17,1,15,5


In [21]:
#读取文章特征
articles = pd.read_csv(data_path+'articles.csv')
articles = reduce_mem(articles)

if click_val is not None:
    all_data = click_trn.append(click_val)
all_data = click_trn.append(click_tst)
all_data = reduce_mem(all_data)

#拼接上文章信息
all_data = all_data.merge(articles, left_on='click_article_id', right_on='article_id')

-- Mem. usage decreased to  5.56 Mb (50.0% reduction),time spend:0.00 min
-- Mem. usage decreased to 46.65 Mb (62.5% reduction),time spend:0.00 min


In [58]:
all_data.shape

(1630633, 13)

#### 4.2.分析一下用户点击时间和点击文章的次数，区分用户活跃度

如果某个用户点击文章之间的时间间隔很小，同时点击的文章次数很多，那么可认为这种用户一般就是活跃用户。下面的函数提供了一种衡量用户活跃度的方式，逻辑如下：

1. 首先根据用户`user_id`分组，对于每个用户，计算点击文章的次数，两两点击文章时间间隔的均值；
2. 把点击次数取倒数和时间间隔的均值统一归一化，然后两者想家合并，该值越小，说明用户越活跃；
3. 如果用户只点击过一次，这时时间间隔均值就会出现空值，对于这种情况最后特征那里会进行区分

衡量用户活跃度的标准并不唯一，可以自己设定。

In [28]:
def active_level(all_data, cols):
    '''
    制作区分用户活跃度的特征
    :param all_data：数据集
    :param cols：用到的特征列
    '''
    data = all_data[cols]
    data.sort_values(['user_id', 'click_timestamp'], inplace=True)
    user_act = pd.DataFrame(data.groupby('user_id', as_index=False)[['click_article_id', 'click_timestamp']].\
                            agg({'click_article_id':np.size, 'click_timestamp':{list}}).values, columns=['user_id', 'click_size', 'click_timestamp'])
    
    #计算用户点击两篇文章的时间间隔的均值
    def time_diff_mean(l):
        if len(l) == 1:
            return 1
        else:
            return np.mean([j-i for i, j in list(zip(l[:-1], l[1:]))])
    
    user_act['time_diff_mean'] = user_act['click_timestamp'].apply(lambda x : time_diff_mean(x))
    
    #点击次数取倒数
    user_act['click_size'] = 1 / user_act['click_size']
    
    #点击次数和点击时间间隔归一化
    user_act['click_size'] = (user_act['click_size'] - user_act['click_size'].min()) / (user_act['click_size'].max() - user_act['click_size'].min())
    user_act['time_diff_mean'] = (user_act['time_diff_mean'] - user_act['time_diff_mean'].min()) / (user_act['time_diff_mean'].max() - user_act['time_diff_mean'].min())
    
    user_act['active_level'] = user_act['click_size'] + user_act['time_diff_mean']
    
    user_act['user_id'] = user_act['user_id'].astype('int')
    del user_act['click_timestamp']
    
    return user_act

In [29]:
user_act_fea = active_level(all_data, ['user_id', 'click_article_id', 'click_timestamp'])

user_act_fea.head()

,user_id,click_size,time_diff_mean,active_level
0,0,0.499466,0.000048,0.499515
1,1,0.499466,0.000048,0.499515
2,2,0.499466,0.000048,0.499515
3,3,0.499466,0.000048,0.499515
4,4,0.499466,0.000048,0.499515


#### 4.3.分析一下文章被点击的时间和被点击的次数，衡量文章热度特征

和上面统计用户热度的方法类似，如果一篇文章在很短的时间间隔之内被点击了很多次，说明文章比热门，实现逻辑如下：
1. 根据文章进行分组，对于每篇文章的用户，计算点击的时间间隔；

2. 将用户的数量取倒数，然后用户的数量和时间间隔归一化，然后相加得到热度特征，该值越小，说明被点击的次数越大且时间间隔越短，文章热度较高。

In [67]:
def hot_level(all_data, cols):
    '''
    制作衡量文章热度的特征
    :param all_data：数据集
    :param cols：用到的特征列
    '''
    data = all_data[cols]
    data.sort_values(['click_article_id', 'click_timestamp'], inplace=True)
    article_hot = pd.DataFrame(data.groupby('click_article_id', as_index=False)[['user_id', 'click_timestamp']].\
                            agg({'user_id':np.size, 'click_timestamp':{list}}).values, columns=['click_article_id', 'user_num', 'click_timestamp'])
    
    #计算用户点击两篇文章的时间间隔的均值
    def time_diff_mean(l):
        if len(l) == 1:
            return 1
        else:
            return np.mean([j-i for i, j in list(zip(l[:-1], l[1:]))])
    
    article_hot['time_diff_mean'] = article_hot['click_timestamp'].apply(lambda x : time_diff_mean(x))
    
    #点击次数取倒数
    article_hot['user_num'] = 1 / article_hot['user_num']
    
    #点击次数和点击时间间隔归一化
    article_hot['user_num'] = (article_hot['user_num'] - article_hot['user_num'].min()) / (article_hot['user_num'].max() - article_hot['user_num'].min())
    article_hot['time_diff_mean'] = (article_hot['time_diff_mean'] - article_hot['time_diff_mean'].min()) / (article_hot['time_diff_mean'].max() - article_hot['time_diff_mean'].min())
    
    article_hot['hot_level'] = article_hot['user_num'] + article_hot['time_diff_mean']
    
    article_hot['click_article_id'] = article_hot['click_article_id'].astype('int')
    del article_hot['click_timestamp']
    
    return article_hot

In [68]:
article_hot_fea = hot_level(all_data, ['user_id', 'click_article_id', 'click_timestamp'])

article_hot_fea.head()

,click_article_id,user_num,time_diff_mean,hot_level
0,3,1,0.0,1
1,69,1,0.0,1
2,84,1,0.0,1
3,94,1,0.0,1
4,125,1,0.0,1


#### 4.4.用户的系列习惯

基于原始的日志数据做一个类似article的那种DataFrame，存放用户特有的信息，主要包括点击习惯，爱好特征之类的，具体如下：
* 用户的设备习惯，这里取最常用的设备（众数）；

* 用户的时间习惯：根据其点击过得历史文章的时间来做一个统计（这个感觉最好是把时间戳里的时间特征的h特征提出来，h是指小时？，看看用户习惯一天的什么时间段点击文章），这先用转换的时间，求均值；

* 用户的爱好特征：对于用户点击的历史文章主题进行用户的爱好判别，更偏向于哪几个主题，这个最好是multi-hot进行编码；

* 用户文章的字数差特征：用户的爱好文章的字数习惯。

##### 4.4.1.用户的设备习惯

In [24]:
def device_fea(all_data, cols):
    '''
    制作用户的设备特征
    :param all_data：数据集
    :param cols：用到的特征列
    '''
    user_device_info = all_data[cols]
    
    #用众数来表示每个用户的设备信息
    user_device_info = user_device_info.groupby('user_id').agg(lambda x : x.value_counts().index[0]).reset_index()
    
    return user_device_info

In [25]:
# 设备特征
device_cols = ['user_id', 'click_environment', 'click_deviceGroup', 'click_os', 'click_country', 'click_region', 'click_referrer_type']

user_device_info = device_fea(all_data, device_cols)

user_device_info.head()

,user_id,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,4,1,17,1,25,2
1,1,4,1,17,1,25,6
2,2,4,3,20,1,25,2
3,3,4,3,2,1,25,2
4,4,4,1,12,1,16,1


##### 4.4.2.用户的时间习惯

In [26]:
def user_time_hob_fea(all_data, cols):
    '''
    制作用户的时间习惯特征
    :param all_data：数据集
    :param cols：用到的特征列
    '''
    user_time_hob_info = all_data[cols]
    
    #先把时间戳进行归一化
    mm = MinMaxScaler()
    user_time_hob_info['click_timestamp'] = mm.fit_transform(user_time_hob_info[['click_timestamp']])
    user_time_hob_info['created_at_ts'] = mm.fit_transform(user_time_hob_info[['created_at_ts']])
    
    user_time_hob_info = user_time_hob_info.groupby('user_id').agg('mean').reset_index()
    
    #user_time_hob1是用户点击文章时间的均值，user_time_hob2是用户点击过的文章的创建时间的均值
    user_time_hob_info.rename(columns={'click_timestamp': 'user_time_hob1', 'created_at_ts': 'user_time_hob2'}, inplace=True)
    
    return user_time_hob_info

In [27]:
user_time_hob_cols = ['user_id', 'click_timestamp', 'created_at_ts']

user_time_hob_info = user_time_hob_fea(all_data, user_time_hob_cols)

user_time_hob_info.head()

,user_id,user_time_hob1,user_time_hob2
0,0,0.343715,0.992865
1,1,0.343618,0.992721
2,2,0.343651,0.992020
3,3,0.343629,0.992774
4,4,0.343702,0.992688


##### 4.4.3.用户的主题爱好

下面先把与用户点击的文章属于的主题转成一个列表，然后在汇总的时候单独制作一个特征：文章的主题如果属于这里面，就是1，否则为0。

In [9]:
def user_cat_hob_fea(all_data, cols):
    '''
    制作用户的主题爱好特征
    :param all_data：数据集
    :param cols：用到的特征列
    '''
    user_category_hob_info = all_data[cols]
    user_category_hob_info = user_category_hob_info.groupby('user_id').agg({list}).reset_index()
    
    user_cat_hob_info = pd.DataFrame()
    user_cat_hob_info['user_id'] = user_category_hob_info['user_id']   
    user_cat_hob_info['cate_list'] = user_category_hob_info['category_id']  
    
    return user_cat_hob_info

In [22]:
user_category_hob_cols = ['user_id', 'category_id']

user_cat_hob_info = user_cat_hob_fea(all_data, user_category_hob_cols)

user_cat_hob_info.head()

,user_id,cate_list
0,0,"[281, 26]"
1,1,"[133, 418]"
2,2,"[297, 43]"
3,3,"[99, 43]"
4,4,"[66, 67]"


##### 4.4.4.用户的字数偏好特征

In [31]:
user_wcou_info = all_data.groupby('user_id')['words_count'].agg('mean').reset_index()

user_wcou_info.rename(columns={'words_count': 'words_hbo'}, inplace=True)

##### 4.4.5.用户的信息特征合并保存

In [32]:
#所有表进行合并
user_info = pd.merge(user_act_fea, user_device_info, on='user_id')
user_info = user_info.merge(user_time_hob_info, on='user_id')
user_info = user_info.merge(user_cat_hob_info, on='user_id')
user_info = user_info.merge(user_wcou_info, on='user_id')

In [33]:
#将合并后的用户特征保存到本地
user_info.to_csv(save_path + 'user_info.csv', index=False)

#### 4.5.将得到的用户特征和文章特征汇总

##### 4.5.1.用户特征直接读入

In [34]:
user_info = pd.read_csv(save_path + 'user_info.csv')

In [35]:
if os.path.exists(save_path + 'trn_user_item_feats_df.csv'):
    trn_user_item_feats_df = pd.read_csv(save_path + 'trn_user_item_feats_df.csv')
    
if os.path.exists(save_path + 'tst_user_item_feats_df.csv'):
    tst_user_item_feats_df = pd.read_csv(save_path + 'tst_user_item_feats_df.csv')

if os.path.exists(save_path + 'val_user_item_feats_df.csv'):
    val_user_item_feats_df = pd.read_csv(save_path + 'val_user_item_feats_df.csv')
else:
    val_user_item_feats_df = None

In [36]:
#将用户特征拼接到之前得到的训练，验证，测试数据上
trn_user_item_feats_df = trn_user_item_feats_df.merge(user_info, on='user_id', how='left')

if val_user_item_feats_df is not None:
    val_user_item_feats_df = val_user_item_feats_df.merge(user_info, on='user_id', how='left')
else:
    val_user_item_feats_df = None

tst_user_item_feats_df = tst_user_item_feats_df.merge(user_info, on='user_id', how='left')

In [37]:
trn_user_item_feats_df.columns

Index(['user_id', 'click_article_id', 'sim0', 'time_diff0', 'word_diff0',
       'sim_max', 'sim_min', 'sim_sum', 'sim_mean', 'score', 'rank', 'label',
       'click_size', 'time_diff_mean', 'active_level', 'click_environment',
       'click_deviceGroup', 'click_os', 'click_country', 'click_region',
       'click_referrer_type', 'user_time_hob1', 'user_time_hob2', 'cate_list',
       'words_hbo'],
      dtype='object')

##### 4.5.2.文章特征直接读入

In [38]:
articles = pd.read_csv(data_path + 'articles.csv')
articles = reduce_mem(articles)

-- Mem. usage decreased to  5.56 Mb (50.0% reduction),time spend:0.00 min


In [40]:
#将文章特征拼接到之前得到的训练，验证，测试数据上
trn_user_item_feats_df = trn_user_item_feats_df.merge(articles, left_on='click_article_id', right_on='article_id')

if val_user_item_feats_df is not None:
    val_user_item_feats_df = val_user_item_feats_df.merge(articles, left_on='click_article_id', right_on='article_id')
else:
    val_user_item_feats_df = None

tst_user_item_feats_df = tst_user_item_feats_df.merge(articles, left_on='click_article_id', right_on='article_id')

##### 4.5.3.构造特征：召回文章的主题是在用户的爱好里面

In [41]:
trn_user_item_feats_df['is_cat_hob'] = trn_user_item_feats_df.apply(lambda x : 1 if x.category_id in set(x.cate_list) else 0, axis=1)

if val_user_item_feats_df is not None:
    val_user_item_feats_df['is_cat_hob'] = val_user_item_feats_df.apply(lambda x : 1 if x.category_id in set(x.cate_list) else 0, axis=1)
else:
    val_user_item_feats_df = None

tst_user_item_feats_df['is_cat_hob'] = tst_user_item_feats_df.apply(lambda x : 1 if x.category_id in set(x.cate_list) else 0, axis=1)

In [42]:
#删除一些不需要的列
del trn_user_item_feats_df['cate_list']

if val_user_item_feats_df is not None:
    del val_user_item_feats_df['cate_list']
else:
    val_user_item_feats_df = None

del tst_user_item_feats_df['cate_list']

del trn_user_item_feats_df['article_id']

if val_user_item_feats_df is not None:
    del val_user_item_feats_df['article_id']
else:
    val_user_item_feats_df = None

del tst_user_item_feats_df['article_id']

##### 4.5.4.保存汇总后的特征

In [43]:
trn_user_item_feats_df.to_csv(save_path + 'trn_user_item_feats_df.csv', index=False)

if val_user_item_feats_df is not None:
    val_user_item_feats_df.to_csv(save_path + 'val_user_item_feats_df.csv', index=False)

tst_user_item_feats_df.to_csv(save_path + 'tst_user_item_feats_df.csv', index=False)